In [6]:
# face recognisation

import numpy as np
import os
import cv2

def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(X,Y,querypoint,k=5):
    vals=[]
    m=X.shape[0]
    #print(m)
    
    for i in range(m):
        d=dist(querypoint,X[i])
        vals.append((d,Y[i]))
        
    vals=sorted(vals)
    vals=vals[:k]
    
    vals=np.array(vals)
    new_vals=np.unique(vals[:,1],return_counts=True)
    #print(new_vals)
    
    index=new_vals[1].argmax()
    pred=new_vals[0][index]
    
    return pred

In [7]:
from sklearn.neighbors import KNeighborsClassifier 

knn =KNeighborsClassifier(n_neighbors=10)



In [8]:
cap=cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
dataset_path='./data/'
face_data=[]
labels=[]

classId=0
names={}
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[classId]=fx[:-4]
        print("loaded "+fx)
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        target=classId*np.ones((data_item.shape[0],))
        classId+=1
        labels.append(target)
        
face_data=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0)

print(face_data)
print(face_labels)




loaded Namrata .npy
loaded pulkit.npy
[[249 223 193 ...  39  39  21]
 [236 216 205 ...  26  23  57]
 [233 219 207 ...  28  31  22]
 ...
 [179 189 201 ... 170 190 196]
 [221 239 255 ... 199 201 190]
 [217 238 255 ... 183 191 202]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [9]:
knn.fit(face_data,face_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [ ]:
while True:
    ret,frame=cap.read()
    if ret ==False:
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h=face
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        out=knn.predict([face_section.flatten()])
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),3)
        
    cv2.imshow("Faces",frame)
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()